최고기온   max_temp   
최저기온   min_temp   
날씨   day_weather
체감온도   feel_temp   
습도   humidity
풍향   wind_direction
풍속   wind_speed
미세먼지   fine_dust
일몰   sunset_time

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                              options=chrome_options)
    return driver


In [2]:
driver = set_chrome_driver()

In [ ]:
url='https://search.naver.com/search.naver?sm=tab_sug.top&where=nx&ssc=tab.nx.all&query=%EB%82%A0%EC%94%A8&oquery=%EC%A0%9C%EC%A3%BC+%EB%82%A0%EC%94%A8&tqi=jfsskspzL8VssZYXKdlssssss1G-440924&acq=%EB%82%A0%EC%94%A8&acr=1&qdt=0&ackey=3xyhxspp'
driver.get(url)
import time

In [30]:
list=['서울','대전','대구','부산','광주','인천','울산','제주']
dlist=[]
for i in list:
    my=driver.find_element(By.CSS_SELECTOR,"#nx_query")
    my.clear()
    my.send_keys(i+' 날씨 \n')
    time.sleep(3)
    print(i)
    html=BeautifulSoup(driver.page_source,'html.parser')
    
    day_weather=html.select_one('span.weather.before_slash').string
    max_temp=html.select_one('.today span.highest').text
    min_temp=html.select_one('.today span.lowest').text
    feel_temp=html.select_one('#main_pack > section.sc_new.cs_weather_new._cs_weather > div > div:nth-child(1) > div.content_wrap > div.open > div:nth-child(1) > div > div.weather_info > div > div._today > div.temperature_info > dl > div:nth-child(1) > dd').string
    humidity=html.select_one('#main_pack > section.sc_new.cs_weather_new._cs_weather > div > div:nth-child(1) > div.content_wrap > div.open > div:nth-child(1) > div > div.weather_info > div > div._today > div.temperature_info > dl > div:nth-child(2) > dd').string
    wind_direction=html.select_one('#main_pack > section.sc_new.cs_weather_new._cs_weather > div > div:nth-child(1) > div.content_wrap > div.open > div:nth-child(1) > div > div.weather_info > div > div._today > div.temperature_info > dl > div:nth-child(3) > dt').string
    wind_speed=html.select_one('#main_pack > section.sc_new.cs_weather_new._cs_weather > div > div:nth-child(1) > div.content_wrap > div.open > div:nth-child(1) > div > div.weather_info > div > div._today > div.temperature_info > dl > div:nth-child(3) > dd').string
    fine_dust=html.select_one('#main_pack > section.sc_new.cs_weather_new._cs_weather > div > div:nth-child(1) > div.content_wrap > div.open > div:nth-child(1) > div > div.weather_info > div > div.report_card_wrap > ul > li:nth-child(1) > a > span').string
    sunset_time=html.select_one('#main_pack > section.sc_new.cs_weather_new._cs_weather > div > div:nth-child(1) > div.content_wrap > div.open > div:nth-child(1) > div > div.weather_info > div > div.report_card_wrap > ul > li.item_today.type_sun > span > span').string
    
    max_temp=int(max_temp.split('온')[-1].replace('°',''))
    min_temp=int(min_temp.split('온')[-1].replace('°',''))
    feel_temp=float(feel_temp.replace('°',''))
    humidity=float(humidity.replace('%',''))
    wind_speed=float(wind_speed.split('m')[0])


    print(max_temp,min_temp)
    print(day_weather,feel_temp,humidity)
    print(wind_direction,wind_speed,fine_dust,sunset_time)
    dlist.append((i,max_temp,min_temp,day_weather,feel_temp,humidity,wind_direction,wind_speed,fine_dust,sunset_time))



서울
12 1
구름많음 11.4 45.0
남서풍 4.3 좋음 17:18
대전
12 2
맑음 12.2 33.0
남서풍 2.0 좋음 17:19
대구
13 3
맑음 12.7 20.0
서풍 2.0 좋음 17:15
부산
15 7
맑음 14.6 28.0
남서풍 2.3 좋음 17:15
광주
13 4
맑음 12.3 38.0
북풍 3.6 좋음 17:24
인천
12 2
맑음 10.7 50.0
서풍 4.9 좋음 17:20
울산
13 4
맑음 13.0 19.0
서풍 3.5 좋음 17:13
제주
16 12
맑음 11.5 61.0
서풍 3.5 좋음 17:29


In [31]:
import pymysql
try:
    conn=pymysql.connect(host='localhost',user='root',password='1234',database='test')
    cur=conn.cursor()

    sql='insert into weather values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    cur.executemany(sql,dlist)
    conn.commit()
    conn.close()
except Exception as err:
    print(err)